In [134]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import argparse
import math
import os
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from PIL import Image
import torch.utils.data as data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision
from __future__ import print_function
import torch.optim as optim
from sklearn.metrics import f1_score
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets
import sys
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [135]:
#used
def parse_option(args):
    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('--print_freq', type=int, default=10,
                        help='print frequency')
    parser.add_argument('--save_freq', type=int, default=50,
                        help='save frequency')
    parser.add_argument('--batch_size', type=int, default=128,
                        help='batch_size')
    parser.add_argument('--num_workers', type=int, default=2,
                        help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=100,
                        help='number of training epochs')
    parser.add_argument('--device', type=str, default='cuda:0')
    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.05,
                        help='learning rate')
    parser.add_argument('--patient_lambda', type=float, default=1,
                        help='learning rate')
    parser.add_argument('--cluster_lambda', type=float, default=1,
                        help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='100',
                        help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1,
                        help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-4,
                        help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='momentum')
    parser.add_argument('--train_csv_path', type=str, default='train data csv')
    parser.add_argument('--test_csv_path', type=str, default='test data csv')
    parser.add_argument('--train_image_path', type=str, default='train data csv')
    parser.add_argument('--test_image_path', type=str, default='test data csv')
    parser.add_argument('--train_clin_path', type=str, default='train clin csv')
    parser.add_argument('--test_clin_path', type=str, default='test clin csv')
    parser.add_argument('--val_csv_path', type=str, default='test clin csv')
    
    parser.add_argument('--parallel', type=int, default=1, help='data parallel')
    parser.add_argument('--ncls', type=int, default=6, help='Number of Classes')
    # model dataset
    parser.add_argument('--model', type=str, default='resnet50')
    parser.add_argument('--dataset', type=str, default='TREX_DME',
                        choices=[ 'OLIVES'], help='dataset')
    parser.add_argument('--mean', type=str, help='mean of dataset in path in form of str tuple')
    parser.add_argument('--std', type=str, help='std of dataset in path in form of str tuple')
    parser.add_argument('--data_folder', type=str, default=None, help='path to custom dataset')
    parser.add_argument('--size', type=int, default=128, help='parameter for RandomResizedCrop')

    # temperature
    parser.add_argument('--temp', type=float, default=0.07,
                        help='temperature for loss function')



    opt = parser.parse_args(args)

    # check if dataset is path that passed required arguments
    if opt.dataset == 'path':
        assert opt.data_folder is not None \
               and opt.mean is not None \
               and opt.std is not None

    # set the path according to the environment
    if opt.data_folder is None:
        opt.data_folder = './datasets/'
    opt.model_path = './save/{}_models'.format(opt.dataset)

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))

    opt.model_name = '{}_lr_{}_decay_{}_bsz_{}_temp_{}'. \
        format(opt.model, opt.learning_rate,
               opt.weight_decay, opt.batch_size, opt.temp)


    opt.save_folder = os.path.join(opt.model_path, opt.model_name)
    if not os.path.isdir(opt.save_folder):
        os.makedirs(opt.save_folder)

    return opt

In [136]:
# Not used
# make a csv file from xlsx
def combine_excel(csv_dir):
    filenames = glob.glob(csv_dir + "/*.xlsx")
    outputxlsx = pd.DataFrame()

    for file in filenames:
        df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
        outputxlsx = outputxlsx.append(df, ignore_index=True)

    outputxlsx.to_csv('test_set_labels.csv',index=False)

# Not used
# to-do any analyzing on the data frames
def analyze_dataframe(csv_dir):
    pass

# Not used
# to-do any image processing to simplify the images
def process_images(csv_dir):
    df = pd.read_csv(csv_dir)

    for i in tqdm(range(0,len(df))):
        path = df.iloc[i,0]
        im = Image.open(path).convert('L')

# Not used
# saves the numpy file into a .csv for submission
def numpy_submission(sub_dir,np_dir):
    np_file  = np.load(np_dir)
    print(len(np_file))
    sub_dir = pd.read_csv(sub_dir)
    print(len(sub_dir))
    for i in range(0,len(sub_dir)):
        sub_dir.iloc[i,1] = np_file[i,0]
        sub_dir.iloc[i, 2] = np_file[i, 1]
        sub_dir.iloc[i, 3] = np_file[i, 2]
        sub_dir.iloc[i, 4] = np_file[i, 3]
        sub_dir.iloc[i, 5] = np_file[i, 4]
        sub_dir.iloc[i, 6] = np_file[i, 5]
    print(sub_dir.head())
    sub_dir.to_csv('baseline_result.csv',index=False)

In [137]:
class OLIVES(data.Dataset):
    def __init__(self,df, img_dir, clinical_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
        self.clinical_dir = pd.read_csv(clinical_dir)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        temp_path = self.df.iloc[idx,0][1:9]
        if temp_path == "TREX DME":
            path = self.img_dir + "/TREX_DME" + self.df.iloc[idx,0]
        else:
            path = self.img_dir  + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        b1 = self.df.iloc[idx,1]
        b2 = self.df.iloc[idx,2]
        b3 = self.df.iloc[idx,3]
        b4 = self.df.iloc[idx, 4]
        b5 = self.df.iloc[idx, 5]
        b6 = self.df.iloc[idx, 6]
        bio_tensor = torch.tensor([b1, b2, b3, b4, b5, b6])
        assert self.df.iloc[idx,0] == self.clinical_dir.iloc[idx,0]
        
        c1 = (self.clinical_dir.iloc[idx,1] - 27) / 73
        c2 = (self.clinical_dir.iloc[idx,2] - 151) / 572
        clinical = torch.tensor([c1, c2])
        return image, bio_tensor, clinical

class RECOVERY(data.Dataset):
    def __init__(self,df, img_dir, transforms):
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        temp_path = self.df.iloc[idx,0][1:9]
        if temp_path == "TREX DME":
            path = self.img_dir + "/TREX_DME" + self.df.iloc[idx,0]
        else:
            path = self.img_dir  + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        return image

class RECOVERY_TEST(data.Dataset):
    def __init__(self,df, img_dir,clinical_dir, transforms): #
        self.img_dir = img_dir
        self.transforms = transforms
        self.df = pd.read_csv(df)
        self.clinical_dir = pd.read_csv(clinical_dir)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        temp_path = self.df.iloc[idx,0][0:9]
        if temp_path == "TREX DME": #changed
            path = self.img_dir + "/TREX_DME" + self.df.iloc[idx,0]
        else:
            path = self.img_dir  + self.df.iloc[idx,0]
        image = Image.open(path).convert("L")
        image = np.array(image)
        image = Image.fromarray(image)
        image = self.transforms(image)
        b1 = self.df.iloc[idx,1]
        b2 = self.df.iloc[idx,2]
        b3 = self.df.iloc[idx,3]
        b4 = self.df.iloc[idx, 4]
        b5 = self.df.iloc[idx, 5]
        b6 = self.df.iloc[idx, 6]
        bio_tensor = torch.tensor([b1, b2, b3, b4, b5, b6])
        
        assert self.df.iloc[idx,0] == self.clinical_dir.iloc[idx,0]
        
        c1 = (self.clinical_dir.iloc[idx,1] - 57) / 39
        c2 = (self.clinical_dir.iloc[idx,2] - 160) / 166

        clinical = torch.tensor([c1, c2])
        return image, bio_tensor, clinical

In [138]:
class MLP(nn.Module):
    # define model elements
    def __init__(self, n_inputs, n_outputs):
        super(MLP, self).__init__()
        self.layer = nn.Sequential(

            # for clinical label only baseline
            nn.Linear(n_inputs, n_inputs),
            nn.LeakyReLU(0.2, True),
            nn.Linear(n_inputs, n_outputs)
        )

    # forward propagate input
    def forward(self, X):
        X = self.layer(X)
        return X

In [139]:
class ResNet(nn.Module):
    """encoder + classifier"""
    def __init__(self, name='resnet50', num_classes=2):
        super(ResNet, self).__init__()

        self.encoder = torchvision.models.resnet50(zero_init_residual=True)
        self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.encoder.fc = nn.Identity()
        self.clin = MLP(2, 8)
        self.fc = MLP(2048+8, num_classes)

    def forward(self, x, c):
        encoder_out = self.encoder(x)
        clinical_out = self.clin(c)
        # Concatenate along the feature dimension
        concat_both = torch.cat([encoder_out, clinical_out], dim=1)
        return self.fc(concat_both)

In [140]:
# create a model and a criterion (loss function) based on passed arguments opt
def set_model(opt):
    device = opt.device
    model = ResNet(name=opt.model,num_classes = opt.ncls)

    criterion = torch.nn.BCEWithLogitsLoss()

    model = model.to(device)
    criterion = criterion.to(device)

    return model, criterion

# used in main
def set_loader(opt):
    # construct data loader
    if opt.dataset == 'OLIVES' or opt.dataset == 'RECOVERY':
        mean = (.1706)
        std = (.2112)
    else:
        raise ValueError('dataset not supported: {}'.format(opt.dataset))

    normalize = transforms.Normalize(mean=mean, std=std)

    # set of transforms to be performed on the training dataset
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=224, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),

        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])
    
    # set of transforms to be performed on the test dataset
    val_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize,
    ])

    # assign paths and create dataset objects
    if opt.dataset =='OLIVES':
        csv_path_train = opt.train_csv_path
        csv_path_val = opt.val_csv_path
        csv_path_test = opt.test_csv_path
        data_path_train = opt.train_image_path
        data_path_test = opt.test_image_path
        clin_path_train = opt.train_clin_path
        clin_path_test = opt.test_clin_path
        
        train_dataset = OLIVES(csv_path_train,data_path_train,clin_path_train ,transforms = train_transform)
        val_dataset = RECOVERY_TEST(csv_path_val,data_path_test, clin_path_test ,transforms = val_transform)
        test_dataset = RECOVERY_TEST(csv_path_test,data_path_test, clin_path_test ,transforms = val_transform) 
    else:
        raise ValueError(opt.dataset)

    # define data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=opt.batch_size, shuffle=True,
        num_workers=opt.num_workers, pin_memory=True)
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True,drop_last=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True,drop_last=False)

    return train_loader, test_loader, val_loader

# used in main
# compute and store metrics during training
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# not used
# calculate the accuracy of model predictions
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

# not used
# Dynamically adjust the learning rate while optimizing
def adjust_learning_rate(args, optimizer, epoch):
    lr = args.learning_rate
    if args.cosine:
        eta_min = lr * (args.lr_decay_rate ** 3)
        lr = eta_min + (lr - eta_min) * (
                1 + math.cos(math.pi * epoch / args.epochs)) / 2
    else:
        steps = np.sum(epoch > np.asarray(args.lr_decay_epochs))
        if steps > 0:
            lr = lr * (args.lr_decay_rate ** steps)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# not used
# gradually increase the learning rate during the initial epochs of training
def warmup_learning_rate(args, epoch, batch_id, total_batches, optimizer):
    if args.warm and epoch <= args.warm_epochs:
        p = (batch_id + (epoch - 1) * total_batches) / \
            (args.warm_epochs * total_batches)
        lr = args.warmup_from + p * (args.warmup_to - args.warmup_from)

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

# used in the end
# creates and configures optimizer for the model
def set_optimizer(opt, model):

    optimizer = optim.SGD(model.parameters(),
                          lr=opt.learning_rate,
                          momentum=opt.momentum,
                          weight_decay=opt.weight_decay)
    return optimizer

# Save the trained model's state, along with relevant training information, to a specified file
def save_model(model, optimizer, opt, epoch, save_file):
    print('==> Saving...')
    state = {
        'opt': opt,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }
    torch.save(state, save_file)
    del state

In [141]:
max_acc=0
max_acc_epoch=0

# used in the end and main
def train_supervised(train_loader, model,criterion, optimizer, epoch, opt):
    """one epoch training"""
    model.train()


    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    device = opt.device
    end = time.time()
    correct_predictions = 0

    for idx, (image, bio_tensor, clinical) in enumerate(train_loader):
        data_time.update(time.time() - end)

        images = image.to(device)
        clinicals = clinical.float().to(device)

        labels = bio_tensor.float()

        labels = labels.to(device)
        bsz = labels.shape[0]

        # compute loss
        

        output = model(images, clinicals)

        
        loss = criterion(output, labels)
        
        predicted_labels = torch.round(torch.sigmoid(output))
        correct_predictions += (predicted_labels == labels).sum().item()

        # update metric
        losses.update(loss.item(), bsz)

        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        total_values = len(train_loader.dataset)*6
        training_accuracy = (correct_predictions / total_values)*100

        # print info
        if (idx + 1) % opt.print_freq == 0:
            print('Train: [{0}][{1}/{2}]\t'.format(epoch, idx + 1, len(train_loader)))
            print(f"Training Accuracy =  {training_accuracy:.2f}%")

            sys.stdout.flush()
        
        global max_acc
        global max_acc_epoch
        if training_accuracy > max_acc:
            max_acc = training_accuracy
            max_acc_epoch = epoch

    return losses.avg


# used in main

def validate(val_loader, model, criterion, opt):
    """Validation"""
    model.eval()

    device = opt.device
    losses = AverageMeter()
    label_list = []
    out_list = []

    with torch.no_grad():
        for idx, (image, bio_tensor, clinical) in enumerate(val_loader):
            images = image.to(device)
            clinicals = clinical.float().to(device)
            labels = bio_tensor.float()
            label_list.append(labels.squeeze().detach().cpu().numpy())
            labels = labels.to(device)
            bsz = labels.shape[0]

            output = model(images, clinicals)
            loss = criterion(output, labels)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())

            losses.update(loss.item(),bsz)
    
    label_array = np.array(label_list)
    out_array = np.array(out_list)
    r = f1_score(label_array, out_array, average='macro')

    return losses.avg, r

def submission_generate(val_loader, model, opt):
    """validation"""
    model.eval()

    device = opt.device
    out_list = []
    with torch.no_grad():
        for idx, (image,  bio_tensor, clinical) in (enumerate(val_loader)):

            images = image.float().to(device)
            clinicals = clinical.float().to(device)

            # forward
            output = model(images, clinicals)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())


    out_submisison = np.array(out_list)
    np.save('output',out_submisison)

# used in main
def sample_evaluation(val_loader, model, opt):
    """validation"""
    model.eval()

    device = opt.device
    out_list = []
    label_list = []
    with torch.no_grad():
        for idx, (image, bio_tensor, clinical) in (enumerate(val_loader)):

            images = image.float().to(device)
            labels = bio_tensor.float()

            labels = labels.float()

            label_list.append(labels.squeeze().detach().cpu().numpy())
            # forward
            output = model(images, clinical)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())

    label_array = np.array(label_list)
    out_array = np.array(out_list)
    f = f1_score(label_array,out_array,average='macro')
    print(f)


def main():
    # access parse_op function
    opt = parse_option(args)

    # build data loader
    train_loader,test_loader, val_loader = set_loader(opt)

    # build model and criterion
    model, criterion = set_model(opt)

    # build optimizer
    optimizer = set_optimizer(opt, model)


    # training routine
    for epoch in range(1, 1 + opt.epochs):
        train_supervised(train_loader, model, criterion, optimizer, epoch, opt)
        
        loss, r = validate(val_loader, model, criterion, opt)
        print('Train epoch {}, f1_score:{:.2f}'.format(epoch, r))
    
    print("Max accuracy: ", max_acc, " at epoch: ", max_acc_epoch)

    submission_generate(test_loader, model, opt)
    #sample_evaluation(test_loader, model, opt)

    save_file = os.path.join(
        opt.save_folder, 'last.pth')
    save_model(model, optimizer, opt, opt.epochs, save_file)

    return
# if __name__ == '__main__':
    # main()

In [142]:
#/kaggle/input/d/salmankhondker/olives-vip-cup-2023/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST

In [143]:
args = ['--batch_size', '128', '--model', "resnet50", '--dataset', 'OLIVES', '--epochs', '25', '--device', 'cuda:0', 
        '--train_image_path', '/kaggle/input/olives-vip-cup-2023/olives/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TRAIN/OLIVES', 
        '--test_image_path', '/kaggle/input/olives-vip-cup-2023/olives/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/', 
        '--test_csv_path', '/kaggle/input/olives-vip-cup-2023/olives/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/test_set_submission_template.csv',
        '--train_csv_path', '/kaggle/input/modified-training-biomarkers/Modified training biomarkers.csv', 
        '--train_clin_path' , '/kaggle/input/clinical/Modified training clinical.csv' ,
        '--test_clin_path', '/kaggle/input/clinical/Modified test clinical.csv',
        '--val_csv_path', '/kaggle/input/test-dataset/Leaked answers.csv']

In [144]:
now_lets_run_our_code = main()

Training Accuracy =  0.73%
Training Accuracy =  1.67%
Training Accuracy =  2.56%
Training Accuracy =  3.42%
Training Accuracy =  4.26%
Training Accuracy =  5.11%
Training Accuracy =  6.02%
Training Accuracy =  6.91%
Training Accuracy =  7.78%
Train: [1][10/73]	
Training Accuracy =  8.66%
Training Accuracy =  9.51%
Training Accuracy =  10.41%
Training Accuracy =  11.34%
Training Accuracy =  12.21%
Training Accuracy =  13.09%
Training Accuracy =  13.97%
Training Accuracy =  14.84%
Training Accuracy =  15.70%
Training Accuracy =  16.59%
Train: [1][20/73]	
Training Accuracy =  17.48%
Training Accuracy =  18.39%
Training Accuracy =  19.29%
Training Accuracy =  20.18%
Training Accuracy =  21.07%
Training Accuracy =  21.95%
Training Accuracy =  22.86%
Training Accuracy =  23.80%
Training Accuracy =  24.67%
Training Accuracy =  25.54%
Train: [1][30/73]	
Training Accuracy =  26.44%
Training Accuracy =  27.33%
Training Accuracy =  28.24%
Training Accuracy =  29.11%
Training Accuracy =  29.96%
Tr

KeyboardInterrupt: 

In [ ]:
z = numpy_submission('/kaggle/input/olives-vip-cup-2023/olives/2023 IEEE SPS Video and Image Processing (VIP) Cup - Ophthalmic Biomarker Detection/TEST/test_set_submission_template.csv','/kaggle/working/output.npy')

In [ ]:
%tb